## Spark Streaming - Trigger

### Rate Source

No **Rate Source** do Spark Structured Streaming, o valor gerado na coluna **`value`** é automaticamente incrementado a partir de 0, ou seja, ele gera valores inteiros sequenciais sem a possibilidade direta de configurar ou definir o valor manualmente.

#### Como funciona o **Rate Source**:
- A coluna **`value`** é um número que começa em 0 e é incrementado automaticamente à medida que o Spark gera mais linhas.
- A coluna **`timestamp`** indica o horário em que cada linha foi gerada.


#### Exemplo de Leitura com `Rate Source`:

In [0]:
spark

In [0]:
stream_df = (
    spark
        .readStream
        .format("rate")
        .option("rowsPerSecond", 50)
        .load()
)

stream_df.printSchema()


**Esquema gerado pelo `Rate Source`:**

```bash
root
 |-- timestamp: timestamp (nullable = true)
 |-- value: long (nullable = true)
```

A coluna **`value`** é um número do tipo `long` gerado automaticamente pelo Spark. Infelizmente, não há um parâmetro nativo no **Rate Source** que permita definir ou controlar manualmente o valor da coluna `value`.


### Exemplo de Trigger - `processingTime`:

- **Rate Source**: A opção `rowsPerSecond` define que 50 linhas de dados são geradas a cada segundo. Os dados contêm uma coluna `timestamp` e uma coluna `value`.
- **Trigger**: O processamento ocorre a cada 5 segundos, criando um micro-batch a cada intervalo.


In [0]:
stream_df = (
    spark
        .readStream
        .format("rate")
        .option("rowsPerSecond", 50)
        .load()
)

In [0]:
query = (
    stream_df
        .writeStream
            .outputMode("append")
            .format("console")
            .trigger(processingTime="5 seconds")
            .start()
)

query.awaitTermination()


---

### Exemplo de Trigger - `once`

O trigger `once` processa todos os dados disponíveis **apenas uma vez** e encerra o stream.


In [0]:
stream_df = (
    spark
        .readStream
        .format("rate")
        .option("rowsPerSecond", 50)
        .load()
)

In [0]:
query = (
    stream_df
        .writeStream
        .outputMode("append")
        .format("console")
        .trigger(once=True)
        .start()
)

query.awaitTermination()


---

### Exemplo de Trigger - `continuous`

O trigger `continuous` processa os dados **continuamente** com baixa latência. Você pode definir um intervalo de checkpoint para garantir que os dados sejam processados assim que chegam.


In [0]:
stream_df = (
    spark
        .readStream
        .format("rate")
        .option("rowsPerSecond", 50)
        .load()
)

In [0]:
# Processar os dados continuamente com baixa latência
query = (
    stream_df
        .writeStream
        .outputMode("append")
        .format("console")
        .trigger(continuous="1 second")  # Latência de 1 segundo
        .start()
)

query.awaitTermination()


---

### Exemplo de Trigger - `availableNow`

O trigger `availableNow` processa **todos os dados disponíveis no momento** e continua aguardando novos dados.


In [0]:
stream_df = (
    spark
        .readStream
        .format("rate")
        .option("rowsPerSecond", 50)
        .load()
)

In [0]:
# Processar todos os dados disponíveis agora e continuar ativo
query = (
    stream_df
        .writeStream
        .outputMode("append")
        .format("console")
        .option("checkpointLocation", "/Volumes/workspace/default/laboratorio-spark/exercicio.04.03/_checkpoint")
        .trigger(availableNow=True)
        .start()
)

query.awaitTermination()


---

### Comparação dos Exemplos com Triggers

| Trigger Type         | Descrição                                                                   | Exemplo                                                                                     |
|----------------------|-----------------------------------------------------------------------------|---------------------------------------------------------------------------------------------|
| **`processingTime`** | Executa em intervalos regulares de tempo. Processa dados a cada N segundos. | Processa os dados a cada 5 segundos, independentemente de novos dados.                      |
| **`once`**           | Processa todos os dados disponíveis uma única vez e encerra o stream.       | Lê todos os dados disponíveis da fonte de taxa e finaliza a execução do stream.             |
| **`continuous`**     | Processa dados continuamente com baixa latência.                            | Processa os dados conforme eles chegam, com uma latência de 1 segundo.                      |
| **`availableNow`**   | Processa os dados disponíveis no momento e aguarda novos dados.             | Processa todos os dados que estão disponíveis agora e continua ativo para novos dados.      |

